In [15]:
import pandas as pd
import requests
import numpy as np
import json
import datetime
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

def run_query(query):  # A simple function to use requests.post to make the API call.
    headers = {'X-API-KEY': 'BQYCaXaMZlqZrPCSQVsiJrKtxKRVcSe4'}
    request = requests.post('https://graphql.bitquery.io/', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        print(request.reason)
        raise Exception('Query failed and return code is {}.{}'.format(request.status_code, query))

query_date = '"2021-12-16", "{}"'.format(str(datetime.date.today()))

# Convert this to JSON

VOLT_ACCOUNTS = {
    # CSP
    "SOL-P": "6Nkc8MEiz3WLz1xthYitmSuy3NGwn7782upRHo2iFmXK",
    "SOL-P-high": "JAoeEG2sfQ1xgXUNVVkJ5mCTCw4SLc6oJafDwf6jTf",
    "BTC-P": "GrB6vbG2WP7eEnbwgxUbBGRMeXYq139jo2o9oW8cNK8f",
    "SOL-P (tsUSDC)": "AQRGh6PU7LzDHvvoPNS7wVVQaCBeftw9kVDAnvuEjbs8",
    "ETH-P": "EA29Xf3HGMtYziw7UKZDUKby7gkoCbXwmiNKwc7z54Ax",
    "LUNA-P": "5kA7FPiB3t2X5s65dK1AoEu5asDjC5d7f5vaB4iY2yrj",
    "MNGO-P": "CVrRw6VtxSjokm2tKmaS5RCuoc9EFjN4wEoov6f2PST6",
#     "SOL-P-Step": ,
#     "SOL-P-Parrot": ,
#     "SOL-P-UXD": ,
    
    "SOL-C": "Hxtb6APfNtf9m8jJjh7uYp8fCTGr9aeHxBSfiPqCrV6G",
    "SOL-C-high": "DNa849drqW19uBV5X9ohpJ5brRGzq856gk3HDRqveFrA",
    "BTC-C": "DA1M8mw7GnPNKU9ReANtHPQyuVzKZtsuuSbCyc2uX2du",
    "mSOL-C": "6asST5hurmxJ8uFvh7ZRWkrMfSEzjEAJ4DNR1is3G6eH",
    "ETH-C": "FThcy5XXvab5u3jbA6NjWKdMNiCSV3oY5AAkvEvpa8wp",
    "FTT-C": "7KqHFuUksvNhrWgoacKkqyp2RwfBNdypCYgK9nxD1d6K",
    "SRM-C": "2P427N5sYcEXvZAZwqNzjXEHsBMESQoLyjNquTSmGPMb",
    "MNGO-C": "B3yakZxwomkmnCxRr8ZmQtiWgtxtVBuCREDFDdAvcCVQ",
    "scnSOL-C": "A5MpyajTy6hdsg3S2em5ukcgY1ZBhxTxEKv8BgHajv1A",
    "SBR-C": "BH7Jg3f97FyeGxsPR7FFskvfqGiaLeUnJ9Ksda53Jj8h",
    "LUNA-C": "5oV1Yf8q1oQgPYuHjepjmKFuaG2Wng9dzTqbSWhU5W2X",
    "RAY-C": "A6XsYxGj9wpqUZG81XwgQJ2zJ3efCbuWSQfnkHqUSmdM",
    "STEP-C": "FiEHDTKT6X7VFwGaUmsm1XXYr8vvkoSR5EqcY4znpefq",
    "AVAX-C": "HLfkEc6E2HEcABaWm3QZSfDvswzXjh1onF9xEzUAtWNS",
}

ACCOUNT_STRING = str(list(VOLT_ACCOUNTS.values())).replace("\'", "\"")

FEES_ADDRESS = '3KjJiWBfaw96qGhysq6Fc9FTxdPgPTNY6shM7Bwfp8EJ'

MASTER_ADDRESS = 'DxMJgeSVoe1cWo1NPExiAsmn83N3bADvkT86dSP1k7WE'

query = """
    query{
    solana(network: solana) {
    transfers(
      date: {between: ["""+ query_date +"""]}
      transferType: {is: transfer}
      any: [{senderAddress: {in: """ + ACCOUNT_STRING + """}}]
    ) {
      amount
      currency {
        symbol
        address
      }
      block {
        timestamp {
          iso8601
        }
      }
      transaction {
        signer
        signature
        innerInstructionsCount
      }
      receiver {
        address
      }
      sender {
        address
      }
    }
  }
}
"""

result = run_query(query)
# convert GraphQL json to pandas dataframe
df = pd.json_normalize(result['data']['solana']['transfers'])
df = df.rename(columns={"block.timestamp.iso8601": "date", "currency.symbol": "symbol", 
                        "transaction.signer":"signer", "currency.address":"cash",
                        "receiver.address":"receiver", "sender.address":"asset", "transaction.signature": "tx_id"})
# Does json_normalize flatten a json file for you??

df["tx_id"] = df["tx_id"].apply(lambda x: "https://solscan.io/tx/{}".format(x))
# if transaction signer & receiver is diff, drop row
df = df.query("receiver == signer").reset_index(drop=True)

def find_key(value, d): 
    for k, v in d.items():
        if value == v:
            return k

df['asset'] = df.asset.apply(lambda x: find_key(x, VOLT_ACCOUNTS))
df['symbol'] = df.apply(lambda x: "tsUSDC" if x.cash == "Cvvh8nsKZet59nsDDo3orMa3rZnPWQhpgrMCVcRDRgip" else x.symbol, axis=1)
df["FeeType"] = df["transaction.innerInstructionsCount"].apply(lambda x: "Withdrawal Fee" if x==1 else "Performance Fee")

In [16]:
z = df.loc[df.receiver=="7cqa7tPYNgWw8eRNDcBosEQxYJ4uKXfDMHFmBhxH3uQN"]

In [17]:
z

,amount,symbol,cash,date,signer,tx_id,transaction.innerInstructionsCount,receiver,asset,FeeType
1446,2.090222,,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,2022-01-25T15:19:40Z,7cqa7tPYNgWw8eRNDcBosEQxYJ4uKXfDMHFmBhxH3uQN,https://solscan.io/tx/5wP9YreHHSsSSTgRrJ3DZigeZScDyXNEBNWQjNgDbyAXCeMiFtr6LYkoJWMrsqzX9DJvPz2NjrmM1b73HoXa24rF,2,7cqa7tPYNgWw8eRNDcBosEQxYJ4uKXfDMHFmBhxH3uQN,SOL-C,Performance Fee
2519,3.969430,,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,2022-02-16T23:29:23Z,7cqa7tPYNgWw8eRNDcBosEQxYJ4uKXfDMHFmBhxH3uQN,https://solscan.io/tx/5YqykYy2pSCgjkZtA4DiFiYBbaKKWjSwNSRUQwjdNmAxqw8MB8VhDXQFhJTJ9zq9u1VvgEMjjQonXF8aEUC7tYYu,2,7cqa7tPYNgWw8eRNDcBosEQxYJ4uKXfDMHFmBhxH3uQN,SOL-C,Performance Fee
5273,6.080488,,7wDh4VCTPwx41kvbLE6fkFgMEjnqw7NpGJvQtNabCm2B,2022-01-29T14:23:42Z,7cqa7tPYNgWw8eRNDcBosEQxYJ4uKXfDMHFmBhxH3uQN,https://solscan.io/tx/2q6mP1kV98ukSEmgUSpkBMXV9knNThYtRH2ZNhoMcwLswU51tGFncvqwBA2DpAj5DUgvc8Kwpm8Kt6MwxCLqQhMh,2,7cqa7tPYNgWw8eRNDcBosEQxYJ4uKXfDMHFmBhxH3uQN,FTT-C,Performance Fee
5728,6.066568,,95sn4kgeJnnBfRCD8S2quu4HS9Y6vb7JDuXrarnmEjYE,2022-01-25T14:57:09Z,7cqa7tPYNgWw8eRNDcBosEQxYJ4uKXfDMHFmBhxH3uQN,https://solscan.io/tx/4A8vRFpHm1zkE1fQ98hXjC8a64TXwVSqCv4J25kRfDsbqrufi8JEoLNGfq9TYLMYEZLyQXd3urVYJCBGwBSphwhL,2,7cqa7tPYNgWw8eRNDcBosEQxYJ4uKXfDMHFmBhxH3uQN,LUNA-C,Performance Fee
5818,8.036131,,95sn4kgeJnnBfRCD8S2quu4HS9Y6vb7JDuXrarnmEjYE,2022-01-29T14:25:39Z,7cqa7tPYNgWw8eRNDcBosEQxYJ4uKXfDMHFmBhxH3uQN,https://solscan.io/tx/5HpzGFo1ZACZ1Vra74WGsHv99wLHCCcJpwZySNiicZrZ2GFYxAhNnLuQB74WrQtYoMviGLFQv8eJTdDzaTGYyWbm,2,7cqa7tPYNgWw8eRNDcBosEQxYJ4uKXfDMHFmBhxH3uQN,LUNA-C,Performance Fee
6266,8.968288,,95sn4kgeJnnBfRCD8S2quu4HS9Y6vb7JDuXrarnmEjYE,2022-03-05T12:35:19Z,7cqa7tPYNgWw8eRNDcBosEQxYJ4uKXfDMHFmBhxH3uQN,https://solscan.io/tx/4Exd7hLSxbzuEWJzAKWb1CcmdN26dTNBBEbjQ3TXLP2N3269WASKQYXGqwoyuVLDt4o4x71wSqocJbhW6nKiEFBt,3,7cqa7tPYNgWw8eRNDcBosEQxYJ4uKXfDMHFmBhxH3uQN,LUNA-C,Performance Fee


In [18]:
z.loc[z.cash=="95sn4kgeJnnBfRCD8S2quu4HS9Y6vb7JDuXrarnmEjYE"].to_csv("./tx.csv")

In [8]:
z.amount.sum()

35.211126991